<a href="https://colab.research.google.com/github/HasibAlMuzdadid/Machine-Learning-and-Deep-Learning-Projects/blob/main/word%20analogy%20and%20debiasing/word_analogy_and_debiasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Word Analogy**

Word embeddings are very computationally expensive to train and most ML practitioners load a pre-trained set of embeddings. Here we'll try our hand at loading, measuring similarity between and modifying pre-trained embeddings. 

In [ ]:
import numpy as np

In [ ]:
# helper function

def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

We will use 50-dimensional [GloVe vectors](https://nlp.stanford.edu/projects/glove/) to represent words.

In [ ]:
# loading word vectors

words, word_to_vec_map = read_glove_vecs("/content/drive/MyDrive/colab files/glove.6B.50d.txt")

- `words`: set of words in the vocabulary.
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.


**Embedding Vectors Versus One-Hot Vectors**

One-hot vectors don't do a good job of capturing the level of similarity between words. This is because every one-hot vector has the same Euclidean distance from any other one-hot vector.

Embedding vectors, such as GloVe vectors, provide much more useful information about the meaning of individual words.  


**Cosine Similarity**

To measure the similarity between two words, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta) $$

* $u \cdot v$ is the dot product (or inner product) of two vectors
* $||u||_2$ is the norm (or length) of the vector $u$
and defined as $ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$

* $\theta$ is the angle between $u$ and $v$. 
* The cosine similarity depends on the angle between $u$ and $v$. 
    * If $u$ and $v$ are very similar, their cosine similarity will be close to 1.
    * If they are dissimilar, the cosine similarity will take a smaller value. 


In [ ]:
# cosine similarity

def cosine_similarity(u, v):
        
        # Arguments:
        # u -> a word vector of shape (n,)          
        # v -> a word vector of shape (n,)

        # Returns:
        # cosine_similarity -> the cosine similarity between u and v 

    
    # Special case. Consider the case u = [0, 0], v=[0, 0]
    if np.all(u == v):
        return 1
    
    # Compute the dot product between u and v 
    dot = np.dot(u, v) 
   
    # Compute the L2 norm of u 
    norm_u = np.sqrt(np.sum(u**2))
    
    # Compute the L2 norm of v 
    norm_v = np.sqrt(np.sum(v**2))
    
    # Avoid division by 0
    if np.isclose(norm_u * norm_v, 0, atol=1e-32):
        return 0
    
    # Compute the cosine similarity 
    cosine_similarity = dot/(norm_u * norm_v)

    
    return cosine_similarity

**Word Analogy**

In the word analogy task, complete this sentence:  
    <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. 

An example is:  <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. 

We're trying to find a word *d* such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner:   
$e_b - e_a \approx e_d - e_c$.
Measuring the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity 

In [ ]:
# complete analogy

def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    
    # Arguments:
    # word_a -> a word, string
    # word_b -> a word, string
    # word_c -> a word, string
    # word_to_vec_map -> dictionary that maps words to their corresponding vectors 
    
    # Returns:
    # best_word ->  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity

    
    # convert words to lowercase
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    # Get the word embeddings e_a, e_b and e_c 
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]

    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output
    
    # loop over the whole word vector set
    for w in words:   
        # to avoid best_word being one the input words, skip the input word_c
        # skip word_c from query
        if w == word_c:
            continue
        

        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  
        cosine_sim = cosine_similarity(e_b - e_a, word_to_vec_map[w] - e_c)
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
        # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word 
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w

        
    return best_word

In [ ]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large')]
for triad in triads_to_try:
    # print('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad, word_to_vec_map)))
    print(f"{triad[0]} -> {triad[1]} :: {triad[2]} -> {complete_analogy(triad[0],triad[1],triad[2], word_to_vec_map)}")

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> smaller


Cosine similarity is a relatively simple and intuitive, yet powerful method we can use to capture nuanced relationships between words.

**Debiasing Word Vectors** 

We'll examine gender biases that can be reflected in a word embedding and explore algorithms for reducing the bias. In addition to learning about the topic of debiasing which will also help hone our intuition about what word vectors are doing. 

First, discovering how the GloVe word embeddings relate to gender. We'll begin by computing a vector $g = e_{woman}-e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. The resulting vector $g$ roughly encodes the concept of "gender". 

We might get a more accurate representation if we compute $g_1 = e_{mother}-e_{father}$, $g_2 = e_{girl}-e_{boy}$ etc and average over them but just using $e_{woman}-e_{man}$ will give good enough results for now.


In [ ]:
g = word_to_vec_map["woman"] - word_to_vec_map["man"]
print(g)

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


Considering the cosine similarity of different words with $g$. What does a positive value of similarity mean versus a negative cosine similarity? 

In [ ]:
print ("List of names and their similarities with constructed vector:")

# girls and boys name
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for w in name_list:
    print(f"{w} : {cosine_similarity(word_to_vec_map[w], g)}")

List of names and their similarities with constructed vector:
john : -0.23163356145973724
marie : 0.315597935396073
sophie : 0.31868789859418784
ronaldo : -0.31244796850329437
priya : 0.17632041839009402
rahul : -0.16915471039231716
danielle : 0.24393299216283895
reza : -0.07930429672199553
katy : 0.2831068659572615
yasmin : 0.23313857767928758


We can see, female first names tend to have a positive cosine similarity with our constructed vector $g$, while male first names tend to have a negative cosine similarity. This is not surprising and the result seems acceptable. 

Now trying with some other words:

In [ ]:
print("Other words and their similarities:")
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print(f"{w} : {cosine_similarity(word_to_vec_map[w], g)}")

Other words and their similarities:
lipstick : 0.2769191625638267
guns : -0.1888485567898898
science : -0.06082906540929701
arts : 0.008189312385880337
literature : 0.06472504433459932
warrior : -0.20920164641125288
doctor : 0.11895289410935041
tree : -0.07089399175478091
receptionist : 0.33077941750593737
technology : -0.13193732447554302
fashion : 0.03563894625772699
teacher : 0.17920923431825664
engineer : -0.0803928049452407
pilot : 0.0010764498991916937
computer : -0.10330358873850498
singer : 0.1850051813649629


It is astonishing how these results reflect certain unhealthy gender stereotypes. We see “computer” is negative and is closer in value to male first names while “literature” is positive and is closer to female first names.

We will reduce the bias of these vectors using an algorithm due to [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520). Some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender-specific, while other words such as "receptionist" or "technology" should be neutralized i.e. not be gender-related. We'll have to treat these two types of words differently when debiasing.


**Neutralize Bias for Non-Gender Specific Words** 

If we're using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction $g$ and the remaining 49 dimensions which is called $g_{\perp}$ here. In linear algebra, we say that the 49-dimensional $g_{\perp}$ is perpendicular (or "orthogonal") to $g$. The neutralization step takes a vector such as $e_{receptionist}$ and zeros out the component in the direction of $g$, giving us $e_{receptionist}^{debiased}$. 


**Implementing Neutralize** 

To remove the bias of words such as "receptionist" or "scientist."

Given an input embedding $e$, we can use the following formulas to compute $e^{debiased}$: 

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g$$
$$e^{debiased} = e - e^{bias\_component}$$



In [ ]:
def neutralize(word, g, word_to_vec_map):
    
        # Arguments:
        # word -> string indicating the word to debias
        # g -> numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        # word_to_vec_map -> dictionary mapping words to their corresponding vectors
    
        # Returns:
        # e_debiased -> neutralized word vector representation of the input "word"

    
    # Selecting word vector representation of "word" using word_to_vec_map
    e = word_to_vec_map[word]
    
    # Computing e_biascomponent using the formula given above
    e_biascomponent = np.dot(e, g)/np.sum(g**2) * g
 
    # Neutralizing e by subtracting e_biascomponent from it 
    # e_debiased should be equal to its orthogonal projection
    e_debiased = e - e_biascomponent

    
    return e_debiased

In [ ]:
e = "receptionist"
print(f"cosine similarity between {e} and g before neutralizing: {cosine_similarity(word_to_vec_map['receptionist'], g)}")

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print(f"cosine similarity between {e} and g after neutralizing: {cosine_similarity(e_debiased, g)}")

cosine similarity between receptionist and g before neutralizing: 0.33077941750593737
cosine similarity between receptionist and g after neutralizing: -2.6832242276243644e-17


**Equalization Algorithm for Gender-Specific Words**

let's apply debiasing to word pairs such as "actress" and "actor". Equalization is applied to pairs of words that we might want to have differ only through the gender property.  Suppose that "actress" is closer to "babysit" than "actor". By applying neutralization to "babysit" we can reduce the gender stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit". The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words are equidistant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$ or from any other work that has been neutralized. 

The derivation of the linear algebra to do this is a bit more complex. Here are the key equations: 

$$ \mu = \frac{e_{w1} + e_{w2}}{2}$$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
$$ 

$$\mu_{\perp} = \mu - \mu_{B} $$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
$$ 
$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
$$


$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {||(e_{w1} - \mu_{\perp}) - \mu_B||_2} $$


$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {||(e_{w2} - \mu_{\perp}) - \mu_B||_2} $$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} $$
$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} $$



In [ ]:
def equalize(pair, bias_axis, word_to_vec_map):
    
    # Arguments:
    # pair -> pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    # bias_axis -> numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    # word_to_vec_map -> dictionary mapping words to their corresponding vectors
    
    # Returns:
    # e_1 -> word vector corresponding to the first word
    # e_2 -> word vector corresponding to the second word

    
    # Step 1: Selecting word vector representation of "word" using word_to_vec_map
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]
    
    # Step 2: Computing the mean of e_w1 and e_w2 
    mu = (e_w1 + e_w2) / 2

    # Step 3: Computing the projections of mu over the bias axis and the orthogonal axis 
    mu_B = np.dot(mu, bias_axis) * bias_axis /np.linalg.norm(bias_axis)**2
    mu_orth = mu - mu_B

    # Step 4: Using equations to compute e_w1B and e_w2B 
    e_w1B = np.dot(e_w1, bias_axis) * bias_axis/np.linalg.norm(bias_axis)**2
    e_w2B = np.dot(e_w2, bias_axis) * bias_axis/np.linalg.norm(bias_axis)**2
        
    # Step 5: Adjusting the Bias part of e_w1B and e_w2B using the formulas given above 
    corrected_e_w1B = np.sqrt(abs(1 - np.linalg.norm(mu_orth)**2)) * (e_w1B - mu_B)/np.linalg.norm((e_w1B - mu_orth) - mu_B)
    corrected_e_w2B = np.sqrt(abs(1 - np.linalg.norm(mu_orth)**2)) * (e_w2B - mu_B)/np.linalg.norm((e_w2B - mu_orth) - mu_B)
    
    # Step 6: Debiasing by equalizing e1 and e2 to the sum of their corrected projections 
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth
                                                              
    
    return e1, e2

In [ ]:
print("cosine similarities before equalizing:")
print(f"cosine_similarity(word_to_vec_map[\'man\'], gender) = {cosine_similarity(word_to_vec_map['man'], g)}")
print(f"cosine_similarity(word_to_vec_map[\'woman\'], gender) = {cosine_similarity(word_to_vec_map['woman'], g)}")
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print(f"cosine_similarity(e1, gender) = {cosine_similarity(e1, g)}")
print(f"cosine_similarity(e2, gender) = {cosine_similarity(e2, g)}")

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map['man'], gender) = -0.11711095765336832
cosine_similarity(word_to_vec_map['woman'], gender) = 0.35666618846270376

cosine similarities after equalizing:
cosine_similarity(e1, gender) = -0.23142942644908526
cosine_similarity(e2, gender) = 0.2314294264490853
